# 2-1 Rの基本

# 解析サマリ

* ノートブック: **`2-1-Rの基本.ipynb`**

  * **カーネル**: Python 3
  * **セル構成**: Markdown 1、Code 1（大きなコードセルに多機能が詰め込まれている）
  * **内容**:

    * Rの基本操作をPythonで模倣（四則演算、ベクトル/行列/配列、DataFrame、リスト、インデックス参照、for文）
    * CSV読込例（`2-1-1-birds.csv`）
    * 乱数とseed固定
    * **NumPyro** + **ArviZ** による単純な正規モデルのベイズ推定（MCMC）と可視化

* データ: **`2-1-1-birds.csv`**

  * 形状: **8行 × 3列**（`species`, `body_length`, `feather_length`）
  * 種別内訳: `crow` 4件、`sparrow` 4件
  * 要約統計（抜粋）

    * `crow`: body\_length 平均 ≈ 54.45、feather\_length 平均 ≈ 102.35
    * `sparrow`: body\_length 平均 ≈ 12.53、feather\_length 平均 ≈ 20.98
  * 全体の相関: `body_length` と `feather_length` の相関 ≈ **0.996**（非常に強い正の相関）

# リファクタリング方針

* **関心の分離**: テーマ（基礎演算、配列/行列、DataFrame、入出力、乱数、ループ、ベイズ推定）ごとに**セル（スニペット）を分割**
* **再現性**: 乱数は\*\*`numpy.random.default_rng`\*\*で扱い、必要に応じてseedを明示
* **堅牢性**: 外部依存（NumPyro/ArviZ/JAX/Graphviz）は**存在チェック**のうえ実行。未導入ならスキップ
* **可読性**: 最低限の関数化、命名の統一、コメント整理
* **互換性**: 元の教材の意図（R操作のPython置換）を保ちつつ、現代的な書き方へ更新

# 実行順の目安

1→2→3→4→5→6→7→8→9 の**順番**でそのまま実行できます（9は外部ライブラリが無ければ自動でスキップします）。

## 1) セットアップ

### （Markdownセル）

* 主要ライブラリの読み込みと、乱数生成器（Generator）を初期化
* 以降のスニペットから参照される **RNG** を1か所で管理
* **依存ライブラリ**（seaborn等）は任意利用のため読み込みに失敗しても全体は継続

In [ ]:
# 01_setup.py
from __future__ import annotations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 任意: seaborn は利用環境により存在しないことがある
try:
    import seaborn as sns  # noqa: F401
except Exception:
    sns = None  # 未導入でも可

# 乱数生成器（推奨: default_rng）
RNG: np.random.Generator = np.random.default_rng(1)

def set_seed(seed: int = 1) -> np.random.Generator:
    """再現性のためのseed設定。共有のRNGを差し替えて返す。"""
    global RNG
    RNG = np.random.default_rng(seed)
    return RNG

## 2) 基本の演算・関数

### （Markdownセル）

* Pythonでの四則演算や整数除算、剰余、累乗
* NumPyの基本関数（平方根、対数など）の例

In [ ]:
# 02_basics_scalars.py
import numpy as np

# 四則演算など
print(1 + 1)     # 加算
print(2 - 1)     # 減算
print(2 * 3)     # 乗算
print(10 / 4)    # 浮動小数の除算
print(10 // 4)   # 整数除算
print(10 % 4)    # 剰余
print(2 ** 3)    # 累乗

# 関数の例
print(np.sqrt(4))
print(np.log(10))

## 3) ベクトル／等差数列

### （Markdownセル）

* Rの c(), `:` 演算子や `seq` に相当するNumPyの作成法
* 配列表示を含む簡単な確認

In [ ]:
# 03_vectors_sequences.py
import numpy as np

vector_1 = np.array([1, 2, 3, 4, 5])
seq_1_to_10 = np.arange(1, 11)   # 1..10
seq_by_2 = np.arange(1, 11, 2)   # 1,3,5,7,9

print(vector_1)
print(seq_1_to_10)
print(seq_by_2)

## 4) 行列／多次元配列（配列・インデックス）

### （Markdownセル）

* Rのmatrix/arrayに相当するNumPy配列と、行名・列名はpandas.DataFrameで表現
* インデックスの例（`iloc`, `loc`）、形状、範囲抽出、行名/列名の取得

In [ ]:
# 04_matrices_arrays.py
import numpy as np
import pandas as pd

# 行列（byrow=TRUE 相当はC順reshapeで近似）
matrix = np.arange(1, 11).reshape(2, 5, order="C")
matrix_df = pd.DataFrame(matrix, index=["Row1", "Row2"], columns=[f"Col{i}" for i in range(1, 6)])
print(matrix_df)

# 3次元配列
array_1 = np.arange(1, 13).reshape(2, 3, 2, order="C")
print(array_1.shape)

# インデックス例
print(matrix_df.iloc[0, 1])        # Row1, Col2
print(array_1[0, 1, 0])            # [first block, second row, first col]
print(matrix_df.iloc[0, :])        # 1行目
print(matrix_df.iloc[:, 0])        # 1列目
print(matrix_df.iloc[0, 1:4])      # スライス
print((matrix_df.index.tolist(), matrix_df.columns.tolist()))
print(matrix_df.loc["Row1", "Col1"])

## 5) DataFrameの基本操作

### （Markdownセル）

* 列アクセス、行数、先頭表示など
* Rのlistに相当するPythonのdict/list例も1つ

In [ ]:
# 05_dataframe_basics.py
import pandas as pd

data_frame_1 = pd.DataFrame({
    "col1": ["A", "B", "C", "D", "E"],
    "col2": [1, 2, 3, 4, 5]
})
print(data_frame_1)

# 行数
print(data_frame_1.shape[0])

# 列・要素アクセス
print(data_frame_1["col2"])
print(data_frame_1["col2"].iloc[1])  # 2番目の要素
print(data_frame_1.head(2))

# Rのlist相当（dict + list）
my_list = {"a": [1, 2, 3], "b": "hello", "c": True}
print(my_list["a"][0])

## 6) CSV読込と簡易EDA（birds）

### （Markdownセル）

* `2-1-1-birds.csv` を読み込む関数を用意（実行環境に応じて相対パス/絶対パスを自動切替）
* 先頭表示、形状、要約統計、種別ごとの集計

In [ ]:
# 06_io_birds_eda.py
import pandas as pd
from pathlib import Path

def load_birds(path: str | Path = "2-1-1-birds.csv") -> pd.DataFrame:
    path = Path(path)
    if not path.exists():
        alt = Path("/mnt/data/2-1-1-birds.csv")
        if alt.exists():
            path = alt
    df = pd.read_csv(path)
    return df

birds = load_birds()
print(birds.head(3))
print("shape:", birds.shape)

# 要約統計（カテゴリ含む）
print(birds.describe(include="all"))

# 種別ごとの集計
#grouped = birds.groupby("species").agg(
#    body_length=("body_length", ["mean", "std", "min", "max", "count"]),
#    feather_length=("feather_length", ["mean", "std"])
#)
grouped = birds.groupby("species").agg(
    mean_body_length   = ("body_length", "mean"),
    std_body_length    = ("body_length", "std"),
    min_body_length    = ("body_length", "min"),
    max_body_length    = ("body_length", "max"),
    count_body_length  = ("body_length", "count"),
    mean_feather_length= ("feather_length", "mean"),
    std_feather_length = ("feather_length", "std"),
)
print(grouped)

# 相関の参考値
print("corr(body_length, feather_length) =", birds[["body_length", "feather_length"]].corr().iloc[0, 1])

## 7) 乱数と再現性（modern API）

### （Markdownセル）

* 乱数は `np.random.default_rng` を推奨
* 同じseedで**同じ結果**になることを確認
  -（参考）古いAPIの `np.random.seed` も簡単に比較

In [ ]:
# 07_randomness.py
import numpy as np

def demo_random(seed: int = 1):
    # modern API: default_rng
    rng = np.random.default_rng(seed)
    a = rng.normal(0, 1)
    b = rng.normal(0, 1)
    # 同じseedで作り直すと再現
    rng = np.random.default_rng(seed)
    c = rng.normal(0, 1)
    d = rng.normal(0, 1)
    print("default_rng:", a, b, c, d)

    # legacy API（教材互換）
    np.random.seed(seed)
    e = np.random.normal(0, 1, 1)
    np.random.seed(seed)
    f = np.random.normal(0, 1, 1)
    print("legacy np.random:", e, f)

demo_random(1)

## 8) forループとベクトル化

### （Markdownセル）

* 素朴なforループの例
* ループでの乱数代入（平均を要素ごとに変える）
* NumPyの**ベクトル化**で同等処理を簡潔に

In [ ]:
# 08_loops_vectorization.py
import numpy as np

# シンプルなforループ
for i in range(1, 4):
    print(i)

# 要素番号を変えながら（ループ版）
rng = np.random.default_rng(1)
result_vec_loop = np.zeros(3)
for i in range(3):
    result_vec_loop[i] = rng.normal(0, 1)
print("loop:", result_vec_loop)

# 平均値を要素ごとに変える（ループ版）
rng = np.random.default_rng(1)
mean_vec = np.array([10.0, 20.0, 30.0])
result_mean_loop = np.zeros_like(mean_vec)
for i in range(mean_vec.size):
    result_mean_loop[i] = rng.normal(mean_vec[i], 1.0)
print("loop with varying mean:", result_mean_loop)

# ベクトル化（推奨）
rng = np.random.default_rng(1)
result_vec_vec = rng.normal(0.0, 1.0, size=3)
result_mean_vec = mean_vec + rng.normal(0.0, 1.0, size=mean_vec.size)
print("vectorized:", result_vec_vec)
print("vectorized with varying mean:", result_mean_vec)

## 9) NumPyroによるベイズ推定（任意：導入済み環境のみ）

### （Markdownセル）

* 正規モデル：

  $$
  y_i \sim \mathcal{N}(\mu,\sigma),\quad 
  \mu \sim \mathcal{N}(0,10),\ 
  \sigma \sim \text{HalfNormal}(5)
  $$
* 依存関係: `jax`, `numpyro`, `arviz`（`graphviz` はモデル可視化にあると便利）
* 未導入なら**自動スキップ**（エラーになりません）
* 実行すると **要約** と **事後分布／トレースの可視化** を表示

In [ ]:
# 09_bayes_numpyro_demo.py
import numpy as np

def run_bayes_demo():
    try:
        import jax
        import jax.random as random
        import numpyro
        import numpyro.distributions as dist
        from numpyro.infer import MCMC, NUTS
        import arviz as az
        import matplotlib.pyplot as plt
    except Exception as e:
        print("NumPyro/JAX/ArviZ が見つからないためベイズ推定はスキップします。", f"詳細: {e}")
        return

    rng_key = random.PRNGKey(0)

    # 観測データ（例）：平均3, 標準偏差1 の正規乱数 50点
    np.random.seed(123)
    y = np.random.normal(loc=3.0, scale=1.0, size=50)
    print(f"観測データ size={y.size}, mean≈{y.mean():.3f}, sd≈{y.std(ddof=1):.3f}")

    # モデル定義
    def model(y=None):
        mu = numpyro.sample("mu", dist.Normal(0, 10))
        sigma = numpyro.sample("sigma", dist.HalfNormal(5))
        with numpyro.plate("N", y.shape[0]):
            numpyro.sample("obs", dist.Normal(mu, sigma), obs=y)

    # モデル可視化（オプション）
    try:
        from numpyro.contrib.render import render_model
        render_model(model, model_args=(y,), render_distributions=True, render_params=True)
    except Exception:
        pass  # 可視化用依存が無い場合は無視

    # 推論（NUTS + MCMC）
    kernel = NUTS(model)
    mcmc = MCMC(kernel, num_warmup=500, num_samples=1000, num_chains=2, progress_bar=False)
    mcmc.run(rng_key, y=y)
    mcmc.print_summary()

    # 可視化（ArviZ）
    idata = az.from_numpyro(mcmc)
    az.plot_posterior(idata, var_names=["mu", "sigma"], hdi_prob=0.95)
    plt.tight_layout(); plt.show()

    az.plot_trace(idata, var_names=["mu", "sigma"])
    plt.tight_layout(); plt.show()

run_bayes_demo()

# 追加メモ（改善点の要約）

* **巨大な1セル**にまとまっていた処理を、**9つの実行単位**へ分割
* 乱数は **`default_rng`** を中心に再現性を担保
* 外部依存は**try-import**で判定し、未導入環境での学習・復習が止まらないよう配慮
* データ入出力では**関数化**してパスの揺れに強く
* ループの教材例は維持しつつ、**ベクトル化**による簡潔な書き方も併記